# Regression Models in R (tips)

In [2]:
library(readr)
library(mechkar)
library(dplyr)
library(ggplot2)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [3]:
df_train <- read.csv("C:/Users/Rita/DataScience/data/BikeSharing/train.csv")
df_test <- read.csv("C:/Users/Rita/DataScience/data/BikeSharing/test.csv")

In [4]:
head(df_train)
dim(df_train)

,id,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,1,1,0,6,0,2,0.344167,0.363625,0.805833,0.1604460,985
2,2,1,1,0,0,0,2,0.363478,0.353739,0.696087,0.2485390,801
3,3,1,1,0,1,1,1,0.196364,0.189405,0.437273,0.2483090,1349
4,4,1,1,0,2,1,1,0.200000,0.212122,0.590435,0.1602960,1562
5,5,1,1,0,3,1,1,0.226957,0.229270,0.436957,0.1869000,1600
6,6,1,1,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,1606


[1] 365  12

In [5]:
head(df_test)
dim(df_test)

,id,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,366,1,1,0,0,0,1,0.370000,0.375621,0.692500,0.192167
2,367,1,1,1,1,0,1,0.273043,0.252304,0.381304,0.329665
3,368,1,1,0,2,1,1,0.150000,0.126275,0.441250,0.365671
4,369,1,1,0,3,1,2,0.107500,0.119337,0.414583,0.184700
5,370,1,1,0,4,1,1,0.265833,0.278412,0.524167,0.129987
6,371,1,1,0,5,1,1,0.334167,0.340267,0.542083,0.167908


[1] 366  11

# EDA

In [6]:
summary(df_train)

       id          season           mnth           holiday      
 Min.   :  1   Min.   :1.000   Min.   : 1.000   Min.   :0.0000  
 1st Qu.: 92   1st Qu.:2.000   1st Qu.: 4.000   1st Qu.:0.0000  
 Median :183   Median :3.000   Median : 7.000   Median :0.0000  
 Mean   :183   Mean   :2.499   Mean   : 6.526   Mean   :0.0274  
 3rd Qu.:274   3rd Qu.:3.000   3rd Qu.:10.000   3rd Qu.:0.0000  
 Max.   :365   Max.   :4.000   Max.   :12.000   Max.   :1.0000  
    weekday        workingday       weathersit         temp        
 Min.   :0.000   Min.   :0.0000   Min.   :1.000   Min.   :0.05913  
 1st Qu.:1.000   1st Qu.:0.0000   1st Qu.:1.000   1st Qu.:0.32500  
 Median :3.000   Median :1.0000   Median :1.000   Median :0.47917  
 Mean   :3.008   Mean   :0.6849   Mean   :1.422   Mean   :0.48666  
 3rd Qu.:5.000   3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.65667  
 Max.   :6.000   Max.   :1.0000   Max.   :3.000   Max.   :0.84917  
     atemp              hum           windspeed            cnt      


In [7]:
df_train$season <- factor(df_train$season)
df_train$mnth <- factor(df_train$mnth)
df_train$holiday <- factor(df_train$holiday)
df_train$weekday <- factor(df_train$weekday)
df_train$workingday <- factor(df_train$workingday)
df_train$weathersit <- factor(df_train$weathersit)


In [8]:
summary(df_train)

       id      season      mnth     holiday weekday workingday weathersit
 Min.   :  1   1:90   1      : 31   0:355   0:52    0:115      1:226     
 1st Qu.: 92   2:92   3      : 31   1: 10   1:52    1:250      2:124     
 Median :183   3:94   5      : 31           2:52               3: 15     
 Mean   :183   4:89   7      : 31           3:52                         
 3rd Qu.:274          8      : 31           4:52                         
 Max.   :365          10     : 31           5:52                         
                      (Other):179           6:53                         
      temp             atemp              hum           windspeed      
 Min.   :0.05913   Min.   :0.07907   Min.   :0.0000   Min.   :0.02239  
 1st Qu.:0.32500   1st Qu.:0.32195   1st Qu.:0.5383   1st Qu.:0.13558  
 Median :0.47917   Median :0.47285   Median :0.6475   Median :0.18690  
 Mean   :0.48666   Mean   :0.46684   Mean   :0.6437   Mean   :0.19140  
 3rd Qu.:0.65667   3rd Qu.:0.61238   3rd Qu.:0.7

In [29]:
df_test$season <- factor(df_test$season)
df_test$mnth <- factor(df_test$mnth)
df_test$holiday <- factor(df_test$holiday)
df_test$weekday <- factor(df_test$weekday)
df_test$workingday <- factor(df_test$workingday)
df_test$weathersit <- factor(df_test$weathersit)

# MODELS

In [9]:
### The error we will use is the RMSE and RMSLE
rmse <- function(y,y_hat) {
    err <- sqrt(sum((y_hat-y)^2,na.rm=T)/length(y))
    return(err)
}

rmsle <- function(y,y_hat) {
    err <- sqrt(sum((log(y_hat+1)-log(y+1))^2,na.rm=T)/length(y))
    return(err)
}


In [10]:
### Table of resulting errors
### Name, Model, RMSE, RMSLE
err_res <- NULL

## Linear Models

In [11]:
## model with only the original variables
mod1 <- lm(cnt ~., data=df_train)
summary(mod1)


Call:
lm(formula = cnt ~ ., data = df_train)

Residuals:
     Min       1Q   Median       3Q      Max 
-2325.73  -280.10    42.23   305.07  2043.37 

Coefficients: (1 not defined because of singularities)
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1921.288    253.914   7.567 3.71e-13 ***
id             -4.305      3.465  -1.242 0.214989    
season2       437.257    189.483   2.308 0.021627 *  
season3       864.707    223.559   3.868 0.000132 ***
season4      1147.515    190.631   6.020 4.58e-09 ***
mnth2         305.041    182.923   1.668 0.096327 .  
mnth3         592.394    260.565   2.274 0.023626 *  
mnth4        1242.748    398.524   3.118 0.001976 ** 
mnth5        2101.132    488.482   4.301 2.23e-05 ***
mnth6        1920.466    586.647   3.274 0.001172 ** 
mnth7        1282.172    695.033   1.845 0.065952 .  
mnth8        1538.101    781.534   1.968 0.049884 *  
mnth9        2103.660    866.619   2.427 0.015731 *  
mnth10       2115.032    971.292   2.1

In [31]:
pred1<-predict(mod1,newdata=df_train)
rmse(df_train$cnt,pred1)
rmsle(df_train$cnt,pred1)
err_res <- rbind(err_res, data.frame(Name="Base Linear regression", Model="mod1", 
                                     RMSE=rmse(df_train$cnt,pred1), 
                                     RMSLE=rmsle(df_train$cnt,pred1)))

Warning message in predict.lm(mod1, newdata = df_train):
"prediction from a rank-deficient fit may be misleading"


[1] 548.0239

Warning message in log(y_hat + 1):
"NaNs produced"


[1] 0.2373054

Warning message in log(y_hat + 1):
"NaNs produced"


In [32]:
pred1t <- predict(mod1,newdata=df_test)

Warning message in predict.lm(mod1, newdata = df_test):
"prediction from a rank-deficient fit may be misleading"


## Desicion trees

In [34]:
library(tree)
library(rpart)

In [35]:
mod3 <- tree(cnt ~., data=df_train)
mod3

node), split, n, deviance, yval
      * denotes terminal node

 1) root 365 692000000 3406  
   2) temp < 0.45875 168 171500000 2274  
     4) season: 1,2 114  49070000 1784  
       8) id < 69.5 68  13330000 1475 *
       9) id > 69.5 46  19660000 2240  
        18) atemp < 0.425806 38  11860000 2062 *
        19) atemp > 0.425806 8    872800 3086 *
     5) season: 4 54  37290000 3308  
      10) weathersit: 3 5   3106000 1504 *
      11) weathersit: 1,2 49  16250000 3492 *
   3) temp > 0.45875 197 121600000 4371  
     6) weathersit: 3 6    580000 2330 *
     7) weathersit: 1,2 191  95210000 4435  
      14) id < 109.5 9   1389000 3076 *
      15) id > 109.5 182  76360000 4502  
        30) weathersit: 1 123  32240000 4684  
          60) atemp < 0.711182 107  22770000 4776 *
          61) atemp > 0.711182 16   2529000 4070 *
        31) weathersit: 2 59  31580000 4123 *

In [37]:
pred3 <- predict(mod3,newdata=df_train)
rmse(df_train$cnt,pred3)
rmsle(df_train$cnt,pred3)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-tree", Model="mod3", 
                                     RMSE=rmse(df_train$cnt,pred3), 
                                     RMSLE=rmsle(df_train$cnt,pred3)))

[1] 534.4596

[1] 0.2419794

In [38]:
mod4 <- rpart(cnt ~., data=df_train)
mod4

n= 365 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 365 691950000 3405.762  
   2) temp< 0.45875 168 171530300 2273.774  
     4) season=1,2 114  49065890 1783.711  
       8) id< 69.5 68  13329020 1474.882 *
       9) id>=69.5 46  19664140 2240.239  
        18) atemp< 0.4258065 38  11864130 2062.184 *
        19) atemp>=0.4258065 8    872784 3086.000 *
     5) season=4 54  37286910 3308.352  
      10) hum>=0.820208 7   8441789 2035.143 *
      11) hum< 0.820208 47  15807650 3497.979 *
   3) temp>=0.45875 197 121560700 4371.112  
     6) hum>=0.8485415 21  24075730 3340.238 *
     7) hum< 0.8485415 176  72505520 4494.114  
      14) id< 109.5 9   1389192 3075.556 *
      15) id>=109.5 167  52029540 4570.563 *

In [39]:
pred4 <- predict(mod4,newdata=df_train)
rmse(df_train$cnt,pred4)
rmsle(df_train$cnt,pred4)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-rpart", Model="mod4", 
                                     RMSE=rmse(df_train$cnt,pred4), 
                                     RMSLE=rmsle(df_train$cnt,pred4)))

[1] 591.7465

[1] 0.2563219

## Random Forest

In [40]:
library(randomForest)
library(ranger)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'ranger'


The following object is masked from 'package:randomForest':

    importance




In [41]:
mod5 <- randomForest(cnt ~., data=df_train)
mod5


Call:
 randomForest(formula = cnt ~ ., data = df_train) 
               Type of random forest: regression
                     Number of trees: 500
No. of variables tried at each split: 3

          Mean of squared residuals: 266792.5
                    % Var explained: 85.93

In [42]:
pred5 <- predict(mod5,newdata=df_train)
rmse(df_train$cnt,pred5)
rmsle(df_train$cnt,pred5)

err_res <- rbind(err_res, data.frame(Name="RandomForest (RF)", Model="mod5", 
                                     RMSE=rmse(df_train$cnt,pred5), 
                                     RMSLE=rmsle(df_train$cnt,pred5)))

[1] 236.9139

[1] 0.1373479

In [43]:
mod6 <- ranger(cnt ~., data=df_train)
mod6

Ranger result

Call:
 ranger(cnt ~ ., data = df_train) 

Type:                             Regression 
Number of trees:                  500 
Sample size:                      365 
Number of independent variables:  11 
Mtry:                             3 
Target node size:                 5 
Variable importance mode:         none 
Splitrule:                        variance 
OOB prediction error (MSE):       250217.9 
R squared (OOB):                  0.868373 

In [44]:
pred6 <- predict(mod6,data=df_train)
#head(pred6)
rmse(df_train$cnt, pred6$predictions)
rmsle(df_train$cnt, pred6$predictions)
err_res <- rbind(err_res, data.frame(Name="RandomForest (ranger)", Model="mod6", 
                                     RMSE=rmse(df_train$cnt,pred6$predictions), 
                                     RMSLE=rmsle(df_train$cnt,pred6$predictions)))

[1] 234.4322

[1] 0.1367568

## XGBoost

In [45]:
install.packages("xgboost")

Installing package into 'C:/Users/Rita/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'xgboost' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Rita\AppData\Local\Temp\Rtmpya5bcq\downloaded_packages


In [46]:
library(xgboost)

Warning message:
"package 'xgboost' was built under R version 4.0.3"

Attaching package: 'xgboost'


The following object is masked from 'package:dplyr':

    slice




In [48]:
train_num <- df_train

In [49]:
test_num <- df_test

In [50]:
train_num$season <- as.numeric(train_num$season)
train_num$mnth <- as.numeric(train_num$mnth)
train_num$holiday <- as.numeric(train_num$holiday)
train_num$weekday <- as.numeric(train_num$weekday)
train_num$workingday <- as.numeric(train_num$weathersit)
train_num$weathersit <- as.numeric(train_num$mnth)

In [51]:
test_num$season <- as.numeric(test_num$season)
test_num$mnth <- as.numeric(test_num$mnth)
test_num$holiday <- as.numeric(test_num$holiday)
test_num$weekday <- as.numeric(test_num$weekday)
test_num$workingday <- as.numeric(test_num$weathersit)
test_num$weathersit <- as.numeric(test_num$mnth)

In [52]:
summary(test_num)

       id            season           mnth           holiday    
 Min.   :366.0   Min.   :1.000   Min.   : 1.000   Min.   :1.00  
 1st Qu.:457.2   1st Qu.:2.000   1st Qu.: 4.000   1st Qu.:1.00  
 Median :548.5   Median :2.500   Median : 7.000   Median :1.00  
 Mean   :548.5   Mean   :2.495   Mean   : 6.514   Mean   :1.03  
 3rd Qu.:639.8   3rd Qu.:3.000   3rd Qu.: 9.750   3rd Qu.:1.00  
 Max.   :731.0   Max.   :4.000   Max.   :12.000   Max.   :2.00  
    weekday        workingday      weathersit          temp       
 Min.   :1.000   Min.   :1.000   Min.   : 1.000   Min.   :0.1075  
 1st Qu.:2.000   1st Qu.:1.000   1st Qu.: 4.000   1st Qu.:0.3477  
 Median :4.000   Median :1.000   Median : 7.000   Median :0.5142  
 Mean   :3.986   Mean   :1.369   Mean   : 6.514   Mean   :0.5041  
 3rd Qu.:6.000   3rd Qu.:2.000   3rd Qu.: 9.750   3rd Qu.:0.6540  
 Max.   :7.000   Max.   :3.000   Max.   :12.000   Max.   :0.8617  
     atemp             hum           windspeed      
 Min.   :0.1017   Min. 

In [54]:
summary(train_num)

       id          season           mnth           holiday         weekday     
 Min.   :  1   Min.   :1.000   Min.   : 1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.: 92   1st Qu.:2.000   1st Qu.: 4.000   1st Qu.:1.000   1st Qu.:2.000  
 Median :183   Median :3.000   Median : 7.000   Median :1.000   Median :4.000  
 Mean   :183   Mean   :2.499   Mean   : 6.526   Mean   :1.027   Mean   :4.008  
 3rd Qu.:274   3rd Qu.:3.000   3rd Qu.:10.000   3rd Qu.:1.000   3rd Qu.:6.000  
 Max.   :365   Max.   :4.000   Max.   :12.000   Max.   :2.000   Max.   :7.000  
   workingday      weathersit          temp             atemp        
 Min.   :1.000   Min.   : 1.000   Min.   :0.05913   Min.   :0.07907  
 1st Qu.:1.000   1st Qu.: 4.000   1st Qu.:0.32500   1st Qu.:0.32195  
 Median :1.000   Median : 7.000   Median :0.47917   Median :0.47285  
 Mean   :1.422   Mean   : 6.526   Mean   :0.48666   Mean   :0.46684  
 3rd Qu.:2.000   3rd Qu.:10.000   3rd Qu.:0.65667   3rd Qu.:0.61238  
 Max.   :3.000   Max

 לוקח את הרשימה שנמצא בהתחלה ומוריד את הרשימה שנמצא בחלק השני, במקרה הזה זה לא רשימה אלה המשתנה המסביר אבל יכול להיות גם רשימה, צריך להוריד את המשתנה המסביר מטסט

In [55]:
nm<-setdiff(names(train_num), "cnt")
nm

[1] "id"         "season"     "mnth"       "holiday"    "weekday"   
 [6] "workingday" "weathersit" "temp"       "atemp"      "hum"       
[11] "windspeed"

In [56]:
summary(train_num)

       id          season           mnth           holiday         weekday     
 Min.   :  1   Min.   :1.000   Min.   : 1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.: 92   1st Qu.:2.000   1st Qu.: 4.000   1st Qu.:1.000   1st Qu.:2.000  
 Median :183   Median :3.000   Median : 7.000   Median :1.000   Median :4.000  
 Mean   :183   Mean   :2.499   Mean   : 6.526   Mean   :1.027   Mean   :4.008  
 3rd Qu.:274   3rd Qu.:3.000   3rd Qu.:10.000   3rd Qu.:1.000   3rd Qu.:6.000  
 Max.   :365   Max.   :4.000   Max.   :12.000   Max.   :2.000   Max.   :7.000  
   workingday      weathersit          temp             atemp        
 Min.   :1.000   Min.   : 1.000   Min.   :0.05913   Min.   :0.07907  
 1st Qu.:1.000   1st Qu.: 4.000   1st Qu.:0.32500   1st Qu.:0.32195  
 Median :1.000   Median : 7.000   Median :0.47917   Median :0.47285  
 Mean   :1.422   Mean   : 6.526   Mean   :0.48666   Mean   :0.46684  
 3rd Qu.:2.000   3rd Qu.:10.000   3rd Qu.:0.65667   3rd Qu.:0.61238  
 Max.   :3.000   Max

In [57]:
X_train <- data.matrix(train_num[,nm])
y_train <- train_num$cnt
mod7 <- xgboost(data=X_train, label = y_train, nrounds=100,print_every_n = 10)

[1]	train-rmse:2614.907227 
[11]	train-rmse:190.945953 
[21]	train-rmse:87.085617 
[31]	train-rmse:52.676472 
[41]	train-rmse:36.509777 
[51]	train-rmse:24.465176 
[61]	train-rmse:15.888309 
[71]	train-rmse:11.345628 
[81]	train-rmse:8.119646 
[91]	train-rmse:5.775094 
[100]	train-rmse:4.264241 


In [58]:
#X_test <- xgb.DMatrix(test1)
X_test <- data.matrix(test_num[,nm])
y_test <- test_num$cnt

pred7 <- predict(mod7,newdata=X_test)
rmse(y_test,pred7)
rmsle(y_test,pred7)
err_res <- rbind(err_res, data.frame(Name="XGBoost", Model="mod7", 
                                     RMSE=rmse(test$cnt,pred7), 
                                     RMSLE=rmsle(test$cnt,pred7)))

[1] NaN

[1] NaN

ERROR: Error in test$cnt: object of type 'closure' is not subsettable


In [51]:
#train1 <- Matrix::sparse.model.matrix(cnt ~ .-1, data = train_num)

In [52]:
#test1 <- Matrix::sparse.model.matrix(cnt ~ .-1, data = test_num)

In [95]:
#X_test <- xgb.DMatrix(test1)

#X_test <- test1
#y_test <- test$cnt

#pred7 <- predict(mod7,newdata=X_test)
#rmse(y_test,pred7)
#rmsle(y_test,pred7)
#err_res <- rbind(err_res, data.frame(Name="XGBoost", Model="mod7", 
                                    # RMSE=rmse(test_num$cnt,pred7), 
                                    # RMSLE=rmsle(test_num$cnt,pred7)))

[1] 591.9582

[1] 0.28685

## kNN 

In [54]:
### adaboost needs that values to be normalized
min_max <- function(x) { (x -min(x))/(max(x)-min(x))   }

In [55]:
X_train <- sapply(data.frame(as.matrix(train1)),min_max)

In [56]:
X_test <- sapply(data.frame(as.matrix(test1)),min_max)

In [57]:
summary(X_train)

       id             season            mnth           holiday       
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.:0.2514   1st Qu.:0.3333   1st Qu.:0.2727   1st Qu.:0.00000  
 Median :0.4945   Median :0.3333   Median :0.4545   Median :0.00000  
 Mean   :0.5018   Mean   :0.4967   Mean   :0.5041   Mean   :0.01961  
 3rd Qu.:0.7459   3rd Qu.:0.6667   3rd Qu.:0.7273   3rd Qu.:0.00000  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :1.00000  
    weekday         workingday       weathersit          temp       
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.1667   1st Qu.:0.0000   1st Qu.:0.2727   1st Qu.:0.2569  
 Median :0.5000   Median :0.0000   Median :0.4545   Median :0.4891  
 Mean   :0.5124   Mean   :0.2176   Mean   :0.5041   Mean   :0.4969  
 3rd Qu.:0.8333   3rd Qu.:0.5000   3rd Qu.:0.7273   3rd Qu.:0.7329  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :1.0000  
     atemp             hum 

In [58]:
#library(class)
#mod8 <- knn(X_train,X_test,cl=train$cnt)

In [87]:
library(class)
mod8 <- knn(train_num,test_num,cl=train_num$cnt)

In [88]:
str(mod8)

 Factor w/ 247 levels "431","506","623",..: 33 20 19 17 21 36 48 10 10 21 ...


In [89]:
pred8 <- as.numeric(as.character(mod8))

rmse(test$cnt,pred8)
rmsle(test$cnt,pred8)
err_res <- rbind(err_res, data.frame(Name="kNN", Model="mod8", 
                                     RMSE=rmse(test$cnt,pred8), 
                                     RMSLE=rmsle(test$cnt,pred8)))

[1] 33.79699

[1] 0.02525251

## SVM

In [61]:
install.packages("liquidSVM")
library(liquidSVM)

Installing package into 'C:/Users/Rita/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'liquidSVM' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Rita\AppData\Local\Temp\RtmpCO0B7L\downloaded_packages


Warning message:
"package 'liquidSVM' was built under R version 4.0.3"


In [62]:
mod9 <- svm(cnt ~., train)

In [63]:
pred9 <- predict(mod9, newdata=test)

rmse(test$cnt,pred9)
rmsle(test$cnt,pred9)
err_res <- rbind(err_res, data.frame(Name="SVM", Model="mod9", 
                                     RMSE=rmse(test$cnt,pred9), 
                                     RMSLE=rmsle(test$cnt,pred9)))

[1] 480.6174

[1] 0.2060643

In [82]:
err_res %>% arrange(RMSLE)

Name,Model,RMSE,RMSLE
<chr>,<chr>,<dbl>,<dbl>
kNN,mod8,33.79699,0.02525251
SVM,mod9,480.61743,0.20606427
Base Linear regression,mod1,633.02244,0.24676782
Base Linear regression,mod1,633.02244,0.24676782
XGBoost,mod7,579.89200,0.28709500
RandomForest (ranger),mod6,580.79116,0.29448315
RandomForest,mod5,597.52156,0.30082613
RandomForest (RF),mod5,597.52156,0.30082613
Decision Trees-tree,mod3,783.25386,0.35420401


In [83]:
test$cnt

[1] 1606 1321 1263 1162 1421 1650 1927  981  986 1416 1098 1360 1712 1530 1746
 [16] 1589 1913 2115 2475 1450 1446 1851  605 2192 2744 3117 2121 1693 3115 1795
 [31] 2808 3126  795 3429 3204 4191 3872 5312 4362 4182 4864 3409 4660 4492 4679
 [46] 4098 4548 3915 5020 4844 4010 4708 5225 5362 5119 4665 4040 5336 5538 5923
 [61] 4541 4332 3846 4302 4866 3785 4602 3820 3805 4758 3542 5115 4484 4713 4785
 [76] 4760 4511 4274 3641 4352 5423 2429 4826 4765 2416 2913 5041 4570 4748 4381
 [91]  627 4068 3649 4035 4205 3368 1817 3053 2566 1495 2792 3071 3613 3485 2594
[106]  705 3322 2743 1317 2999

In [15]:
nm<-setdiff(names(df_train), "cnt")
nm

[1] "id"         "season"     "mnth"       "holiday"    "weekday"   
 [6] "workingday" "weathersit" "temp"       "atemp"      "hum"       
[11] "windspeed"

In [16]:
df_train1<-df_train[,nm]

In [17]:
library(class)
id<-1:nrow(df_test)
cnt<-knn(df_train1,df_test,cl=df_train1$cnt)

ERROR: Error in knn(df_train1, df_test, cl = df_train1$cnt): 'train' and 'class' have different lengths


In [18]:
id<-366:732

In [20]:
library(liquidSVM)
mod91 <- svm(cnt ~., df_train)

Warning message:
"package 'liquidSVM' was built under R version 4.0.3"


In [22]:
pred91<- predict(mod91, newdata=df_test)

In [25]:
id<-366:731
cnt<-pred91

In [123]:
rmse(df_test$cnt,pred91)
rmsle(df_test$cnt,pred91)

[1] NaN

[1] NaN

In [26]:
dft<-data.frame(id=id, cnt=cnt)
dft

id,cnt
<int>,<dbl>
366,3103.540
367,2421.814
368,2146.626
369,2031.483
370,3061.659
371,3419.342
372,3501.235
373,3007.181
374,2452.967


In [28]:
write.csv(dft, file='results_svm_31102020.csv',row.names=FALSE)